In [1]:
from glob import glob
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import json
import random

In [2]:
csv_t2020_df = pd.read_csv('/home/llm-hackathon/Downloads/fixed-data-csv/all-reports-available-xrays-through2020-final.csv')
csv_a2020_df = pd.read_csv('/home/llm-hackathon/Downloads/fixed-data-csv/all-reports-available-xrays-after2020-final.csv')
csv_ids_through_2020 = csv_t2020_df['AccessionId']
csv_ids_after2020 = csv_a2020_df['AccessionId']
csv_t2020_df_indexed = csv_t2020_df.set_index('AccessionId')

In [3]:
img_fp = '/data/UCSD_cxr/jpg/'
img_ids = glob('*.jpg', root_dir=img_fp)
img_ids.sort()
img_acc_ids = [int(re.search(r'[0-9]+',img_id)[0]) for img_id in img_ids]

In [4]:
print('# Img Acc Ids, # ids thru 2020, # ids after 2020, # img in thru 2020, # img in after 2020')
len(img_acc_ids), len(csv_ids_through_2020), len(csv_ids_after2020), len(set(csv_ids_through_2020).intersection(set(img_acc_ids))), len(set(csv_ids_after2020).intersection(set(img_acc_ids))), 

# Img Acc Ids, # ids thru 2020, # ids after 2020, # img in thru 2020, # img in after 2020


(112839, 59965, 66495, 44189, 61928)

In [5]:
missing_ids = list(set(img_acc_ids) - set(csv_ids_through_2020).intersection(set(img_acc_ids)))
missing_ids

[53870598,
 53870599,
 53870600,
 53346313,
 54001678,
 53870607,
 54001680,
 53870609,
 53346326,
 53870614,
 53870617,
 54001690,
 53477406,
 53870627,
 53870629,
 53346342,
 54525993,
 54001706,
 53346346,
 54001710,
 53870640,
 53477425,
 53346353,
 54263860,
 53608501,
 53477430,
 53870648,
 53739577,
 53608504,
 53608506,
 54001727,
 53739583,
 53870658,
 53870660,
 54004934,
 54001735,
 53608522,
 54001742,
 53739599,
 53346390,
 53346391,
 53477466,
 53477468,
 53477469,
 56754268,
 53477472,
 53477483,
 53477484,
 53870703,
 53346416,
 54001777,
 53477490,
 54001778,
 54001780,
 54001779,
 53346417,
 53346423,
 54001786,
 54001787,
 54001788,
 54001789,
 54001790,
 54001791,
 54001792,
 53477504,
 54001794,
 54001795,
 54001796,
 54001797,
 54001798,
 53346437,
 53477514,
 54001802,
 53346443,
 53614175,
 53739663,
 53739664,
 53346448,
 53346451,
 53870740,
 54001822,
 53346462,
 53608609,
 54001828,
 54001834,
 54001835,
 53739691,
 53346474,
 53608619,
 43647150,
 53346490,

In [6]:
def generate_json(img_ids, save_json_fp, data_fp, question):
    json_output = []

    for img_id in tqdm(img_ids):
        img_acc_id = int(re.search(r'[0-9]+', img_id)[0])
        patient_id = img_id.split('.')[0]
        if img_acc_id in csv_t2020_df_indexed.index:
            prompt = csv_t2020_df_indexed.loc[img_acc_id, 'prompt'] + question
            answer = csv_t2020_df_indexed.loc[img_acc_id, 'answer']

            patient_data = {
                "id": patient_id,
                "image": data_fp + img_id,
                "conversations": [
                    {
                        "from": "human",
                        "value": f"{prompt}"
                    },
                    {
                        "from": "gpt",
                        "value": f"{answer}"

                    }

                ]
            }

            json_output.append(patient_data)

    with open(save_json_fp, 'w') as f:
        json.dump(json_output, f, indent=4)

    print('Dumped JSON')

In [7]:
# Generate train json
# save_json_fp = '/home/llm-hackathon/LLaVA/data/train_patient_finding_impression.json' 
# data_fp = '/home/llm-hackathon/LLaVA/data/jpg/'
# question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
# generate_json(train_img_ids, save_json_fp, data_fp, question)

In [8]:
# Generate test json 
# save_json_fp = '/home/llm-hackathon/LLaVA/data/test_patient_finding_impression.json' 
# data_fp = '/home/llm-hackathon/LLaVA/data/jpg/'
# question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
# generate_json(test_img_ids, save_json_fp, data_fp, question)

In [9]:
t2020 = pd.read_csv('/data/UCSD_cxr/through2020_dropna_formatted.csv')
a2020 = pd.read_csv('/data/UCSD_cxr/after2020_dropna_formatted.csv')
t2020['AccessionId'] = t2020['AccessionId'].astype(str)
a2020['AccessionId'] = a2020['AccessionId'].astype(str)
t2020 = t2020.set_index('AccessionId')
a2020 = a2020.set_index('AccessionId')
train_accn = pd.read_csv('/data/UCSD_cxr/train_accn.csv')['Accession'].astype(str)
test_accn = pd.read_csv('/data/UCSD_cxr/test_accn.csv')['Accession'].astype(str)

# some report text are missing, so we drop them
t2020.dropna(inplace=True)
a2020.dropna(inplace=True)

In [10]:
# bools 
t2020_fu_bools= t2020['answer'].str.contains('FINDINGS:\nUnknown')
t2020_iu_bools = t2020['answer'].str.contains('IMPRESSION: Unknown')

t2020_unknown_bools = t2020_fu_bools | t2020_iu_bools

a2020_fu_bools= a2020['answer'].str.contains('FINDINGS:\nUnknown')
a2020_iu_bools = a2020['answer'].str.contains('IMPRESSION: Unknown')

a2020_unknown_bools = a2020_fu_bools | a2020_iu_bools

In [11]:
t2020_fu = t2020[t2020_fu_bools] # fu = findings unknown
t2020_iu = t2020[t2020_iu_bools] # iu = impression unknown

a2020_fu = a2020[a2020_fu_bools]
a2020_iu = a2020[a2020_iu_bools]

In [12]:
print("a2020 shape:", a2020.shape[0])
print("a2020_fu shape:", a2020_fu.shape[0])
print("a2020_iu shape:", a2020_iu.shape[0])
print("t2020 shape:", t2020.shape[0])
print("t2020_fu shape:", t2020_fu.shape[0])
print("t2020_iu shape:", t2020_iu.shape[0])

a2020 shape: 88852
a2020_fu shape: 6022
a2020_iu shape: 199
t2020 shape: 99536
t2020_fu shape: 7545
t2020_iu shape: 7255


In [13]:
a2020['answer'].str.contains('Unknown').sum()

6027

In [14]:
t2020['answer'].str.contains('Unknown').sum()

7916

In [15]:
# when findings are unknown, extract impression
# t2020_fu['ReportText'].str.extract(r'(?s)(FINDINGS/.*)(IMPRESSION:.*?)(?:CONCURRENT|$)')
# the nans have no findings
findings_t2020_fu = t2020_fu['ReportText'].str.extract(r'(?s)FINDINGS(.*?)(?=IMPRESSION:|Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)
impression_t2020_fu = t2020_fu['ReportText'].str.extract(r'(?s)IMPRESSION:(.*?)(?=Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)

findings_t2020_fu = findings_t2020_fu.apply(lambda x: 'FINDINGS:' + x if type(x) == str else x)
impression_t2020_fu = impression_t2020_fu.apply(lambda x: 'IMPRESSION:' + x if type(x) == str else x)

In [16]:
findings_a2020_fu = a2020_fu['ReportText'].str.extract(r'(?s)FINDINGS(.*?)(?=IMPRESSION:|Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)
impression_a2020_fu = a2020_fu['ReportText'].str.extract(r'(?s)IMPRESSION:(.*?)(?=Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)

findings_a2020_fu = findings_a2020_fu.apply(lambda x: 'FINDINGS:' + x if type(x) == str else x)
impression_a2020_fu = impression_a2020_fu.apply(lambda x: 'IMPRESSION:' + x if type(x) == str else x)

In [17]:
findings_a2020_iu = a2020_iu['ReportText'].str.extract(r'(?s)FINDINGS(.*?)(?=IMPRESSION:|Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)
impression_a2020_iu = a2020_iu['ReportText'].str.extract(r'(?s)IMPRESSION:(.*?)(?=Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)

findings_a2020_iu = findings_a2020_iu.apply(lambda x: 'FINDINGS:' + x if type(x) == str else x)
impression_a2020_iu = impression_a2020_iu.apply(lambda x: 'IMPRESSION:' + x if type(x) == str else x)

In [18]:
findings_t2020_iu = t2020_iu['ReportText'].str.extract(r'(?s)FINDINGS(.*?)(?=IMPRESSION:|Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)
impression_t2020_iu = t2020_iu['ReportText'].str.extract(r'(?s)IMPRESSION:(.*?)(?=Signed by:|\Z)')[0].str.strip().fillna('').apply(lambda x: np.nan if len(x) < 10 else x)

findings_t2020_iu = findings_t2020_iu.apply(lambda x: 'FINDINGS:' + x if type(x) == str else x)
impression_t2020_iu = impression_t2020_iu.apply(lambda x: 'IMPRESSION:' + x if type(x) == str else x)

In [19]:
new_answer_t2020_fu = pd.concat([findings_t2020_fu, impression_t2020_fu], axis=1).apply(lambda x: '\n'.join(x.dropna()) if x.dropna().size > 0 else np.nan, axis=1)
new_answer_t2020_iu = pd.concat([findings_t2020_iu, impression_t2020_iu], axis=1).apply(lambda x: '\n'.join(x.dropna()) if x.dropna().size > 0 else np.nan, axis=1)
new_answer_a2020_fu = pd.concat([findings_a2020_fu, impression_a2020_fu], axis=1).apply(lambda x: '\n'.join(x.dropna()) if x.dropna().size > 0 else np.nan, axis=1)
new_answer_a2020_iu = pd.concat([findings_a2020_iu, impression_a2020_iu], axis=1).apply(lambda x: '\n'.join(x.dropna()) if x.dropna().size > 0 else np.nan, axis=1)

In [20]:
t2020_filled_missing = new_answer_t2020_fu.combine_first(new_answer_t2020_iu).dropna()


In [21]:
a2020_filled_missing = new_answer_a2020_fu.reset_index().groupby('AccessionId').last()[0].combine_first(new_answer_a2020_iu.groupby('AccessionId').last().dropna()).dropna()

In [22]:
t2020.loc[t2020_filled_missing.index, 'answer'] = t2020_filled_missing
a2020.loc[a2020_filled_missing.index, 'answer'] = a2020_filled_missing

In [23]:
t2020['answer'].str.contains('Unknown').sum()

46

In [24]:
a2020['answer'].str.contains('Unknown').sum()

182

In [25]:
a2020

,FalseID,DocumentTitle,DocumentDate,Context,Age,Sex,Author,ReportText,prompt,answer
AccessionId,,,,,,,,,,
53333541,1567456,X-RAY CHEST SINGLE VIEW,1/23/2021,Outpatient,52,Male,"Jacobs, Kathleen",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Jacobs, Kathleen \nCLINICAL HISTORY: D...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53205565,28204320,X-RAY CHEST SINGLE VIEW,12/17/2020,Inpatient,79,Female,"Kligerman, Seth",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Kligerman, Seth \nCLINICAL HISTORY: Ne...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53993975,28240024,X-RAY CHEST FRONTAL AND LATERAL,9/2/2021,Inpatient,70,Male,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST FRONTAL AND LAT...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Rule o...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53513416,28254880,X-RAY CHEST SINGLE VIEW,3/28/2021,Emergency,62,Female,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Sepsis...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53662146,28260488,X-RAY CHEST SINGLE VIEW,5/17/2021,Outpatient,81,Female,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Chest ...",FINDINGS:\nLines and Tubes: Dual-chamber ICD i...
...,...,...,...,...,...,...,...,...,...,...
54167443,472010408,X-RAY CHEST FRONTAL AND LATERAL,10/7/2021,Outpatient,23,Male,"Horowitz, Michael",EXAM DESCRIPTION: X-RAY CHEST FRONTAL AND LAT...,"AUTHOR: Horowitz, Michael \nCLINICAL HISTORY: ...","FINDINGS:\nDevices, lines and Tubes: None M..."
53726745,472054544,X-RAY CHEST SINGLE VIEW,6/7/2021,Outpatient,25,Female,"Rajpoot, Ravi",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Rajpoot, Ravi \nCLINICAL HISTORY: Feve...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53727052,736753880,X-RAY CHEST FRONTAL AND LATERAL,6/7/2021,Outpatient,24,Male,"Rajpoot, Ravi",EXAM DESCRIPTION: X-RAY CHEST FRONTAL AND LAT...,"AUTHOR: Rajpoot, Ravi \nCLINICAL HISTORY: Dysp...",FINDINGS:\nLines and Tubes: None Mediastinum:...


In [26]:
# drop rows with duplicate ids
a2020_len = a2020.shape[0]
t2020_len = t2020.shape[0]

a2020.reset_index(inplace=True)
t2020.reset_index(inplace=True)

a2020.drop_duplicates(subset='AccessionId', inplace=True)
t2020.drop_duplicates(subset='AccessionId', inplace=True)

print('t2020 duplicates:', t2020_len - t2020.shape[0])
print('a2020 duplicates:', a2020_len - a2020.shape[0])

t2020 duplicates: 0
a2020 duplicates: 196


In [27]:
a2020.set_index('AccessionId', inplace=True)
t2020.set_index('AccessionId', inplace=True)

In [28]:
a2020.head()

,FalseID,DocumentTitle,DocumentDate,Context,Age,Sex,Author,ReportText,prompt,answer
AccessionId,,,,,,,,,,
53333541,1567456,X-RAY CHEST SINGLE VIEW,1/23/2021,Outpatient,52,Male,"Jacobs, Kathleen",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Jacobs, Kathleen \nCLINICAL HISTORY: D...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53205565,28204320,X-RAY CHEST SINGLE VIEW,12/17/2020,Inpatient,79,Female,"Kligerman, Seth",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Kligerman, Seth \nCLINICAL HISTORY: Ne...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53993975,28240024,X-RAY CHEST FRONTAL AND LATERAL,9/2/2021,Inpatient,70,Male,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST FRONTAL AND LAT...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Rule o...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53513416,28254880,X-RAY CHEST SINGLE VIEW,3/28/2021,Emergency,62,Female,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Sepsis...",FINDINGS:\nLines and Tubes: None Mediastinum:...
53662146,28260488,X-RAY CHEST SINGLE VIEW,5/17/2021,Outpatient,81,Female,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Chest ...",FINDINGS:\nLines and Tubes: Dual-chamber ICD i...


In [29]:
a2020['answer'].str.contains('Unknown').sum()

93

In [30]:
t2020['answer'].str.contains('Unknown').sum()

46

In [31]:
a2020 = a2020[~a2020['answer'].str.contains('Unknown')]
t2020 = t2020[~t2020['answer'].str.contains('Unknown')]

In [32]:
def contains_date(report):
    # Regular expression pattern to match dates in mm/dd/yyyy format
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    
    # Search for the pattern in the report
    if re.search(date_pattern, report):
        return True  # The report contains a date
    else:
        return False  # The report does not contain a date

Drop reports with dates because those are comparing to other reports (which will just confuse LLava more)

In [33]:
a2020['answer'].apply(contains_date).sum()

4792

In [34]:
t2020['answer'].apply(contains_date).sum()

7573

In [35]:
t2020 = t2020[~t2020['answer'].apply(contains_date)]

In [36]:
a2020 = a2020[~a2020['answer'].apply(contains_date)]

Remove CONCURRENT SUPERVISION from reports (don't want llava to copy and repeat this information)

In [37]:
print(a2020[a2020['answer'].str.contains('CONCURRENT')]['answer'].iloc[0])

FINDINGS:
Lines and Tubes: None  Mediastinum: The cardiomediastinal silhouette is unremarkable. No lymphadenopathy is appreciated.  Lungs: Lungs are well expanded. Streaky bibasal opacities, left greater than right.  Pleura: No left pleural effusion. The right costophrenic sulcus is not imaged. No pneumothorax.  Bones and soft tissues: Partially visualized ORIF hardware of the right humerus without acute hardware complication. Multiple lateral right rib healed fracture deformities.    CONCURRENT SUPERVISION:  I have reviewed the images and agree with the resident interpretation.           Preliminary created by: Besser, Alexandra

IMPRESSION: Streaky bibasal opacities which may be on the basis of atelectasis. No definite focal consolidation.


In [38]:
import pandas as pd
import re

def clean_report(report):
    # Remove "CONCURRENT SUPERVISION" section and "Preliminary created by:", but keep "IMPRESSION"
    cleaned_report = re.sub(r"CONCURRENT SUPERVISION:.*?IMPRESSION:", "IMPRESSION:", report, flags=re.DOTALL)
    return cleaned_report

# Apply the cleaning function to each report in the 'answer' column
a2020['answer'] = a2020['answer'].apply(clean_report)
t2020['answer'] = t2020['answer'].apply(clean_report)


In [39]:
# drop the annoying stragglers (there is only 3 anyway)
t2020 = t2020[~t2020['answer'].str.contains('CONCURRENT')]

In [40]:
t2020['prompt']

AccessionId
44067089    AUTHOR: Kligerman, Seth \nCLINICAL HISTORY: Co...
44067059    AUTHOR: Kligerman, Seth \nCLINICAL HISTORY: Co...
44066941    AUTHOR: Ladd, William \nCLINICAL HISTORY: Stat...
44066704    AUTHOR: Naheedy, John \nCLINICAL HISTORY: The ...
44066777    AUTHOR: Kligerman, Seth \nCLINICAL HISTORY: Lu...
                                  ...                        
53256733    AUTHOR: Hsiao, Albert \nCLINICAL HISTORY: Foll...
53257183    AUTHOR: Hsiao, Albert \nCLINICAL HISTORY: Ches...
53257251    AUTHOR: Rastogi, Monika \nCLINICAL HISTORY: At...
53257270    AUTHOR: Rastogi, Monika \nCLINICAL HISTORY: Ch...
53257291    AUTHOR: Rastogi, Monika \nCLINICAL HISTORY: Sh...
Name: prompt, Length: 91914, dtype: object

In [41]:
# Generate initial json
data_fp = img_fp
json_output = []
save_json_fp = '/home/llm-hackathon/LLaVA/raymond/030424patient_finding_impression.json'
question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
# question = '\nBased on AUTHOR and CLINICAL HISTORY information, suppose you were the named specialist AUTHOR, provide me a detailed report reading this chest X-ray as closely resembling the style of this AUTHOR as possible.'
#question = 'Write a report for the given chest x-ray. It should contain a clear findings and impression section.  Be explicit for any abnormalities or normal findings.'
c_t2020 = 0
c_a2020 = 0
i = 0
j = 0
k = 0
train_set = set(train_accn)

for img_id in tqdm(img_ids):
    img_acc_id = str(re.search(r'[0-9]+', img_id)[0])
    patient_id = img_id.split('.')[0]
    # if img_acc_id in t2020.index:
    #     i +=1
    # if img_acc_id in train_set:
    #     j+=1
    # if img_acc_id in a2020.index:
    #     k+=1
    if img_acc_id in t2020.index and img_acc_id in train_set:
        c_t2020 += 1
        prompt = t2020.loc[img_acc_id, 'prompt'] + question
        answer = t2020.loc[img_acc_id, 'answer']

    elif img_acc_id in a2020.index and img_acc_id in train_set:
        c_a2020 += 1
        prompt = a2020.loc[img_acc_id, 'prompt'] + question
        answer = a2020.loc[img_acc_id, 'answer']
    else:
        continue

    patient_data = {
        "id": patient_id,
        "image": data_fp + img_id,
        "conversations": [
            {
                "from": "human",
                "value": f"{prompt}"
            },
            {
                "from": "gpt",
                "value": f"{answer}"

            }

        ]
    }
    json_output.append(patient_data)
print(f'---Through 2020 Completed, Found {c_t2020} Patients---')
print(f'---After 2020 Completed, Found {c_a2020} Patients---')


with open(save_json_fp, 'w') as f:
    json.dump(json_output, f, indent=4)

print('Dumped JSON')

100%|██████████| 112839/112839 [00:01<00:00, 75514.71it/s]


---Through 2020 Completed, Found 39527 Patients---
---After 2020 Completed, Found 56594 Patients---
Dumped JSON


In [42]:
len(test_set.intersection(set([str(id) for id in img_acc_ids])))

NameError: name 'test_set' is not defined

In [43]:
# Generate questions file.
data_fp = img_fp
save_json_fp = '/home/llm-hackathon/LLaVA/raymond/030424_test_patient_finding_impression.jsonl'
test_set = set(test_accn)
#question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?\nAlso, identify from the following list of conditions--[Edema, Pneumonia, Pneumothorax, Atelectasis, Lung Lesion, Consolidation, Cardiomegaly, Pleural Effusion, Enlarged Cardiom, Lung Opacity]--and tell me which one(s) this X-ray shows.'
#question = '\nBased on AUTHOR and CLINICAL HISTORY information, suppose you were the named specialist AUTHOR, provide me a detailed report reading this chest X-ray as closely resembling the style of this AUTHOR as possible.'
question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
#question = '\nProvide a detailed report given the chest X-RAY. Do not compare with prior X-RAY, only describe what you see in the given xray.'
#question = 'Write a report for the given chest x-ray. It should contain a clear findings and impression section.  Be explicit for any abnormalities or normal findings.'
c_t2020 = 0
c_a2020 = 0
with open(save_json_fp, 'w') as f:

    for img_id in tqdm(img_ids):
        img_acc_id = str(re.search(r'[0-9]+', img_id)[0])
        patient_id = img_id.split('.')[0]
        if img_acc_id in t2020.index and img_acc_id in test_set:
            c_t2020 += 1
            prompt = t2020.loc[img_acc_id, 'prompt'] + question
        elif img_acc_id in a2020.index and img_acc_id in test_set:
            c_a2020 += 1
            prompt = a2020.loc[img_acc_id, 'prompt'] + question
        else:
            continue
        patient_data = {
            "question_id": patient_id,
            "image": data_fp + img_id,
            "text": f"{prompt}",
            "category": "",
        }
        f.write(json.dumps(patient_data) + "\n")


print('Dumped JSON')
c_t2020, c_a2020

100%|██████████| 112839/112839 [00:00<00:00, 378231.19it/s]

Dumped JSON


(859, 1229)

In [44]:
#  CREATE CSV WITH ANSWERS FOR TEST SET
test_set = set(test_accn)
intersection_a2020 = a2020.index.intersection(test_set)
intersection_t2020 = t2020.index.intersection(test_set)

In [45]:
pd.concat([a2020.loc[intersection_a2020], t2020.loc[intersection_t2020]]).to_csv('./030424_test_patient_finding_impression_answers.csv')